In [1]:
import pandas as pd
from dateutil import parser
import datetime

In [128]:
fanpov = pd.read_csv('data/collaborationPatterns-autobiography-meta-info.csv', sep='\t',lineterminator='\n',
                     index_col=None, header=0)

/Users/christinedekock/opt/anaconda3/envs/ml_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [139]:
blp = pd.read_csv('data/blp_articles.csv')

In [144]:
blp.cl_from.values

array([ 6368510,  6225163,  8634662, ..., 48923146, 44542564, 65365484])

In [153]:
fanpov.page_id

0         18192043.0
1         18192043.0
2         18192043.0
3         18192043.0
4         18192043.0
             ...    
756244    31137615.0
756245    31137615.0
756246    31137615.0
756247    31137615.0
756248    31137615.0
Name: page_id, Length: 756249, dtype: float64

In [138]:
fanpov.columns

Index(['event_timestamp', 'page_title', 'page_id', 'revision_id',
       'revision_is_identity_reverted', 'revision_minor_edit',
       'revision_text_bytes', 'revision_first_identity_reverting_revision_id',
       'revision_seconds_to_identity_revert', 'event_user_is_anonymous',
       'has_template', 'event_comment'],
      dtype='object')

In [129]:
groups = fanpov.groupby('page_title')

In [130]:
groups['event_timestamp'].count().describe()

count     4239.000000
mean       178.397971
std        435.898558
min          3.000000
25%         48.000000
50%         85.000000
75%        165.000000
max      10712.000000
Name: event_timestamp, dtype: float64

In [131]:
def get_edits_pre_tag(data):
    ix = data['has_template'].idxmax()
    if ix > data.index.min():
        ix = ix - 1
    
    return data[:ix]

In [132]:
pre_edit = fanpov.groupby('page_title').apply(get_edits_pre_tag)
pre_edit.reset_index(drop=True, inplace=True)
pre_edit = pre_edit.sort_values(by=['page_id','revision_id'], ascending=True)
pre_edit['event_timestamp'] = pd.to_datetime(pre_edit['event_timestamp'])

In [136]:
pre_edit.page_title.nunique()

4239

In [133]:
# group by page
TIMEFRAME_DAYS = 7

def article_age_years(page_revisions):
    diff = page_revisions.event_timestamp.max() - page_revisions.event_timestamp.min()
    diff = diff // datetime.timedelta(days=365.2425)
    return diff

def curr_size(page_revisions):
    return page_revisions.iloc[-1].revision_text_bytes

def frac_minor_edits(page_revisions):
    return page_revisions.revision_minor_edit.mean()

def last_edit_size(page_revisions):
    edits = page_revisions[-2:].revision_text_bytes.values
    return edits[0]/edits[1]

def num_revisions(page_revisions):
    return len(page_revisions)
               
def frac_recent_revisions(page_revisions, days=TIMEFRAME_DAYS):
    start_time = page_revisions.event_timestamp.max() - datetime.timedelta(days=days)
    recent = page_revisions.query('event_timestamp > @start_time')    
    return len(recent) / len(page_revisions)

In [134]:
pages = pre_edit.groupby('page_title')
pre_edit['time_to_respond'] = pages['event_timestamp'].diff().dt.days.abs().fillna(0)
pre_edit['time_responded_to'] = pages['event_timestamp'].diff(-1).dt.days.abs().fillna(0)
pre_edit['edit_size'] = pages['revision_text_bytes'].diff().fillna(pre_edit.revision_text_bytes)
pre_edit['next_edit_size'] = pages['revision_text_bytes'].diff()[1:].tolist() + [0]
pre_edit['revision_is_identity_revert'] = pre_edit.revision_id.isin(pre_edit.revision_first_identity_reverting_revision_id)

In [135]:
# group by page and user

def num_revisions(user_revisions):
    return len(user_revisions)

def frac_edits_reverted(user_revisions):
    return user_revisions.revision_is_identity_reverted.mean()

def frac_reverted_others(user_revisions):
    return user_revisions.revision_is_identity_revert.mean()

def mean_revision_size(user_revisions):
    return user_revisions.edit_size.mean()

def size_of_edits_after(user_revisions):
    return user_revisions.next_edit_size.mean()
    
def time_to_respond(user_revisions):
    return user_revisions.time_to_respond.mean()
    
def time_reponded_to(user_revisions):
    return user_revisions.time_responded_to.mean()

## these are all just column means; easier to just do them at once

def user_revision_feat(user_revisions):
    return user_revisions[['revision_is_identity_reverted','revision_is_identity_revert','edit_size',
                          'next_edit_size','time_to_respond','time_responded_to']].mean()